https://github.com/openai/CLIP/blob/dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1/notebooks/Prompt_Engineering_for_ImageNet.ipynb#L942
https://arxiv.org/abs/2103.00020

In [1]:
%pip install open_clip_torch
%pip install ftfy regex tqdm
%pip install git+https://github.com/openai/CLIP.git
import torch
import clip
from PIL import Image
import os
import open_clip
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch
import os
import numpy as np
from IPython.display import clear_output
import time
import clip
import itertools


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-uf7wdput
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-uf7wdput
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


/home/lnidogon/miniconda3/envs/projektr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_folder_path = "images/"
#image_folder_path = "images/" #for testing locally

In [3]:
torch.cuda.is_available()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
def analyise(image_name, options):
  image = preprocess(Image.open(image_folder_path + image_name).resize((224, 224))).unsqueeze(0).to('cuda')
  text = tokenizer(options).to('cuda')
  with torch.no_grad(), torch.amp.autocast('cuda'):
      image_features = model.encode_image(image).to('cuda')
      text_features = model.encode_text(text).to('cuda')
      image_features /= image_features.norm(dim=-1, keepdim=True)
      text_features /= text_features.norm(dim=-1, keepdim=True)
      text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
  print("Label probs:", list(zip(text_probs.tolist()[0], options)))
  return text_probs.tolist()[0]

def analyise2(image_name, text, path):
  image = preprocess(Image.open(path + image_name).resize((224, 224))).unsqueeze(0).to('cuda')
  with torch.no_grad(), torch.amp.autocast('cuda'):
      image_features = model.encode_image(image).to('cuda')
      text_features = model.encode_text(text).to('cuda')
      image_features /= image_features.norm(dim=-1, keepdim=True)
      text_features /= text_features.norm(dim=-1, keepdim=True)
      text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
  return text_probs.tolist()[0]


def analyise2_no_resize(image_name, text, path):
  image = preprocess(Image.open(path + image_name)).unsqueeze(0).to('cuda')
  with torch.no_grad(), torch.amp.autocast('cuda'):
      image_features = model.encode_image(image).to('cuda')
      text_features = model.encode_text(text).to('cuda')
      image_features /= image_features.norm(dim=-1, keepdim=True)
      text_features /= text_features.norm(dim=-1, keepdim=True)
      text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
  return text_probs.tolist()[0]

def analyise3(image, text):
  image = preprocess(image).unsqueeze(0).to('cuda')
  with torch.no_grad(), torch.amp.autocast('cuda'):
      image_features = model.encode_image(image).to('cuda')
      text_features = model.encode_text(text).to('cuda')
      image_features /= image_features.norm(dim=-1, keepdim=True)
      text_features /= text_features.norm(dim=-1, keepdim=True)
      text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
  return text_probs.tolist()[0]
  print("Label probs:", list(zip(text_probs.tolist()[0], options)))

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
model = model.to('cuda')
model.eval()
tokenizer = open_clip.get_tokenizer('ViT-B-32')
image_list = ["cat.jpg", "dog.jpg", "diagram.jpg"]
for image_name in image_list:
  analyise(image_name, ["a dog", "a cat", "a diagram"])


/home/lnidogon/miniconda3/envs/projektr/lib/python3.11/site-packages/open_clip/factory.py:380: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


Label probs: [(0.00460777385160327, 'a dog'), (0.9950019717216492, 'a cat'), (0.0003902353346347809, 'a diagram')]
Label probs: [(0.9982434511184692, 'a dog'), (0.0005877210642211139, 'a cat'), (0.0011688228696584702, 'a diagram')]
Label probs: [(0.0014317580498754978, 'a dog'), (0.0005434340564534068, 'a cat'), (0.9980248212814331, 'a diagram')]


**Sada ću probati klasifikaciju na različitim slikama koje meni zvuče zanimlijvo**

# **Slika borove šume**

In [6]:
analyise("pine_forest.jpg", ["pine forest", "forest", "woods", "zagorje", "oak forest", "gravity falls", "virovitica"])
analyise("pine_forest.jpg", ["forest", "woods", "zagorje", "oak forest", "gravity falls", "virovitica"])
print("")

Label probs: [(0.9440404176712036, 'pine forest'), (0.03082401119172573, 'forest'), (0.01099064014852047, 'woods'), (3.1047222819324816e-06, 'zagorje'), (0.01411226112395525, 'oak forest'), (9.268988833355252e-06, 'gravity falls'), (2.0245332052581944e-05, 'virovitica')]
Label probs: [(0.5491234064102173, 'forest'), (0.19887931644916534, 'woods'), (5.6180990213761106e-05, 'zagorje'), (0.25140702724456787, 'oak forest'), (0.0001677254622336477, 'gravity falls'), (0.00036634609568864107, 'virovitica')]



Možemo vidjeti da kada je ponuđen točan odgovor (pine forest) sa sigurnošću ga izabire, ali kada ga nema najsigurniji je za forest što je također točno.

# **Slika mravarice**

In [7]:
analyise("velvet_ant.jpg", ["ant", "wingless wasp", "wasp", "velvet ant", "red ant", "fire ant", "cow killer"])
analyise("velvet_ant.jpg", ["ant", "wingless wasp", "wasp", "red ant", "fire ant", "cow killer"])
print("")

Label probs: [(0.047870539128780365, 'ant'), (0.00278639979660511, 'wingless wasp'), (0.006086056120693684, 'wasp'), (0.7971160411834717, 'velvet ant'), (0.1321747899055481, 'red ant'), (0.013931121677160263, 'fire ant'), (3.507559449644759e-05, 'cow killer')]
Label probs: [(0.2362024039030075, 'ant'), (0.01374862901866436, 'wingless wasp'), (0.030029768124222755, 'wasp'), (0.6521757245063782, 'red ant'), (0.06767312437295914, 'fire ant'), (0.00017038648365996778, 'cow killer')]



Ovdje možemo vidjeti da točno prepoznaje kada je ponuđena opcija velvet ant, ali ne radi najbolje kada se makne opcije. I cow killer i wingless wasp su točne klasifikacije mravarice no svejedno odabire s najvećom sigurnošću opciju "red ant" što nije točno (iako istina je da je to najvjerojatniji opis koji bi netko neupoznat s mravaricama dao)

# **Slika Velike Gorice**

In [8]:
analyise("velika_gorica.jpg", ["velika gorica", "glavni grad Zagrebačke županije", "zagreb", "velika mlaka", "metropola"])
analyise("velika_gorica.jpg", ["velika gorica", "sisak", "karlovac", "kopenhagen", "malmo", "tolouse", "hrvatska"])
print("")

Label probs: [(0.07352733612060547, 'velika gorica'), (0.034731850028038025, 'glavni grad Zagrebačke županije'), (0.8818590044975281, 'zagreb'), (0.0015023526502773166, 'velika mlaka'), (0.008379449136555195, 'metropola')]
Label probs: [(0.0861009731888771, 'velika gorica'), (0.6070709824562073, 'sisak'), (0.14419199526309967, 'karlovac'), (0.01862064003944397, 'kopenhagen'), (0.13335579633712769, 'malmo'), (0.007523419335484505, 'tolouse'), (0.0031362276058644056, 'hrvatska')]



Za sliku Velike Gorice klasifica ju kao Zagreb, kada se ponude internacionalni gradovi i dalje ne prepoznaju goricu i klasificira je kao sisak

# **Slika Oca Goriota**

In [9]:
analyise("goriot.jpg", ["old goriot", "vautrin", "mrs. voquer", "honore de balzac", "bach", "raskoljnjikov"])
analyise("goriot.jpg", ["portrait of old goriot", "portrait of vautrin", "portrait of mrs. voquer", "portrait of honore de balzac", "portrait of bach", "portrait of raskoljnjikov"])
print("")

Label probs: [(0.40004169940948486, 'old goriot'), (0.07061155140399933, 'vautrin'), (0.0021322842221707106, 'mrs. voquer'), (0.13610734045505524, 'honore de balzac'), (0.33164623379707336, 'bach'), (0.05946088954806328, 'raskoljnjikov')]
Label probs: [(0.41581836342811584, 'portrait of old goriot'), (0.07691866159439087, 'portrait of vautrin'), (0.005073335487395525, 'portrait of mrs. voquer'), (0.16540111601352692, 'portrait of honore de balzac'), (0.30421918630599976, 'portrait of bach'), (0.032569412142038345, 'portrait of raskoljnjikov')]



Vidimo da prepoznaje u obe situacije sliku oca goriota, ali ovdje je također bitno primijetiti da specificiranje da se radi o portretu dodatno povećava sigurnost.

# **Slika kipa Davida**

In [10]:
analyise("david.jpg", ["david", "david statue", "moses statue", "petrarca statue", "dante statue", "boccacio statue", "abraham lincoln"])
analyise("david.jpg", ["david statue in italy", "david statue in rome", "david statue in florence", "david statue louvre", "david statue in london"])
print("")

Label probs: [(0.002925089094787836, 'david'), (0.8238182067871094, 'david statue'), (0.007469476200640202, 'moses statue'), (0.026071030646562576, 'petrarca statue'), (0.07905943691730499, 'dante statue'), (0.06061697006225586, 'boccacio statue'), (3.9813465264160186e-05, 'abraham lincoln')]
Label probs: [(0.3307494521141052, 'david statue in italy'), (0.3205733597278595, 'david statue in rome'), (0.2829050123691559, 'david statue in florence'), (0.05233216658234596, 'david statue louvre'), (0.013440011069178581, 'david statue in london')]



Sliku kipa Davida je dobro prepoznao te također i to da se nalazi u Firenci.

# **Slika Kahoot pitanja**

In [19]:
analyise("kahoot.jpg", ["red answer is correct", "blue answer is correct", "yellow answer is correct", "green answer is correct"])
analyise("kahoot.jpg", ["red answer is not correct", "blue answer is not correct", "yellow answer is not correct", "green answer is not correct"])
print("")

Label probs: [(0.2015543431043625, 'red answer is correct'), (0.14292344450950623, 'blue answer is correct'), (0.3885061740875244, 'yellow answer is correct'), (0.26701614260673523, 'green answer is correct')]
Label probs: [(0.22071199119091034, 'red answer is not correct'), (0.1858581155538559, 'blue answer is not correct'), (0.3313279151916504, 'yellow answer is not correct'), (0.2621019780635834, 'green answer is not correct')]



Vidimo da CLIP ne može najbolje odgovoriti na ovakvo pitanje (točan odgovor je crveno)

# **Slika haljine s nepoznatim bojama**

In [21]:
analyise("multicolor_dress.jpg", ["white and gold dress", "blue and black dress"])
print("")

Label probs: [(0.4493926465511322, 'white and gold dress'), (0.5506073832511902, 'blue and black dress')]



Mislio sam da će možda biti blizu ali CLIP potvrđuje da je crno plav. Ovo je popularna slika oko čije boje postoji dvojba, CLIP je jako siguran da je plavo crna.

# **Slika slike na kojoj nema ničega**

In [22]:
analyise("name_one_thing.jpg", ["something", "nothing", "plushy", "shirt", "abacus", "paper towel", "a crumbled up shirt"])
print("")

Label probs: [(0.3474450707435608, 'something'), (0.23144832253456116, 'nothing'), (0.012655685655772686, 'plushy'), (0.0663110539317131, 'shirt'), (0.003799925558269024, 'abacus'), (0.09954468160867691, 'paper towel'), (0.23879528045654297, 'a crumbled up shirt')]



Zanimljiv rezultat jer prepoznaje nešto, ali također daje dosta veliku vjerojatnosti tome da ne vidi ništa.

# **Slika Mr. Beasta**

In [23]:
analyise("mr_beast.jpg", ["mr beast", "smile", "millionare", "billionare", "pewdiepie", "logan paul"])
analyise("mr_beast.jpg", ["picture of a millionaire", "picture of a youtuber", "picture of mr beast", "picture of an athlete", "picture of a mechanic"])
print("")

Label probs: [(0.9900130033493042, 'mr beast'), (0.0004539437941275537, 'smile'), (0.0003822590224444866, 'millionare'), (0.002492642030119896, 'billionare'), (0.004045956768095493, 'pewdiepie'), (0.0026122662238776684, 'logan paul')]
Label probs: [(0.0034763291478157043, 'picture of a millionaire'), (0.0014951504999771714, 'picture of a youtuber'), (0.9944868683815002, 'picture of mr beast'), (0.0002898481616284698, 'picture of an athlete'), (0.00025182441459037364, 'picture of a mechanic')]



Vidimo da je Mr Beast jako dobro prepoznan

# **Drugi modeli?**

In [24]:
model, _, preprocess = open_clip.create_model_and_transforms('convnext_base_w', pretrained='laion2b_s13b_b82k')
model.eval()
model.to('cuda')
tokenizer = open_clip.get_tokenizer('convnext_base_w')
analyise("velvet_ant.jpg", ["ant", "wingless wasp", "wasp", "velvet ant", "red ant", "fire ant", "cow killer"])
analyise("velvet_ant.jpg", ["ant", "wingless wasp", "wasp", "red ant", "fire ant", "cow killer"])
print("")


Label probs: [(0.004700520075857639, 'ant'), (0.001188477617688477, 'wingless wasp'), (2.3907168724690564e-05, 'wasp'), (0.98380047082901, 'velvet ant'), (0.009060489013791084, 'red ant'), (0.0012262039817869663, 'fire ant'), (2.6037005884660402e-09, 'cow killer')]
Label probs: [(0.29016271233558655, 'ant'), (0.07336461544036865, 'wingless wasp'), (0.0014757876051589847, 'wasp'), (0.5593032240867615, 'red ant'), (0.07569346576929092, 'fire ant'), (1.607262021252609e-07, 'cow killer')]



Vidimo da ovaj model koji je predtreniran na manjem datasetu ima također "netočnije" rezultate.

In [26]:
model, preprocess = open_clip.create_model_from_pretrained('hf-hub:apple/DFN5B-CLIP-ViT-H-14-384')
tokenizer = open_clip.get_tokenizer('ViT-H-14')
model.eval()
model.to('cuda')
analyise("velvet_ant.jpg", ["ant", "wingless wasp", "wasp", "velvet ant", "red ant", "fire ant", "cow killer"])
analyise("velvet_ant.jpg", ["ant", "wingless wasp", "wasp", "red ant", "fire ant", "cow killer"])
print("")

Label probs: [(1.3419023161986843e-05, 'ant'), (5.3909043344901875e-05, 'wingless wasp'), (3.866630322590936e-07, 'wasp'), (0.9999127388000488, 'velvet ant'), (1.696323488431517e-05, 'red ant'), (2.6013963179138955e-06, 'fire ant'), (4.0182593363802255e-11, 'cow killer')]
Label probs: [(0.1517259031534195, 'ant'), (0.619136393070221, 'wingless wasp'), (0.004440760239958763, 'wasp'), (0.19481991231441498, 'red ant'), (0.029876600950956345, 'fire ant'), (4.6149037302711804e-07, 'cow killer')]



Ovaj model koji je predtreniran na značajno većem datasetu jako sigurno prepoznaje da se radi o slici mravarice te također prepoznaje da se radi o vrsti beskrile ose.

# **Zaključci:**
Klasifikacijski modeli dobro prepoznaju objekte koji se mogu opisati, npr. mravarica ili boje, može se primijetiti da apstraktnije pojmove poput točnog odgovora na kahoot kvizu ne može dobro prepoznati. Također kao i očekivano modeli koji su trenirani na većim datasetovima bolje i sigurnije odlučuju te posjeduju veću moć prepoznavanja apstraktnijih pojmova poput alternativnih naziva.

In [6]:
import torch
import os
import numpy as np
from IPython.display import clear_output

model, preprocess = clip.load("ViT-B/32")
model.to('cuda')
model.eval()
tokenizer = open_clip.get_tokenizer('ViT-B-32')
dataset_folder_path = "CUB_200_2011/CUB_200_2011/"
f = open(dataset_folder_path + "classes.txt")
s = f.read()
class_names = [x.split(' ')[1] for x in s.split('\n')[:-1]]
def analyiseData(n):
  correct_answer = 0
  correct_classifications = 0
  incorrect_classifications = 0
  text = tokenizer(class_names).to('cuda')
  for class_name in class_names:
    arr = os.listdir(os.path.join(dataset_folder_path, "images", class_name))
    m = 0
    for image_name in arr:
      m += 1
      print("Currently classifying " + image_name, "  Done: ", (correct_classifications + incorrect_classifications), "/", n * len(class_names))
      values = np.array(analyise2_no_resize(image_name, text, dataset_folder_path +  "images/" + class_name + "/"))
      #print(list(zip(class_names, values)))
      if np.argmax(values) == correct_answer:
          correct_classifications += 1
      else:
          incorrect_classifications += 1
      clear_output(wait=True)
      if n == m:
        break
    #print(correct_answers)
    #print(incorrect_answers)
    correct_answer += 1
  print("Accuracy on ", n, " images from every class: ", correct_classifications/(correct_classifications + incorrect_classifications))

In [13]:
analyiseData(1)

Accuracy on  1  images from every class:  0.45


Pokrenut je test na 200 različitih slika i accuracy dobiven je 45%

In [15]:
analyiseData(-1)

Accuracy on  -1  images from every class:  0.3951476077366814


Accuracy na cijelom datasetu je 39.51%

In [16]:

model, preprocess = clip.load("ViT-B/32")
model.to('cuda')
model.eval()
tokenizer = open_clip.get_tokenizer('ViT-B-32')
dataset_folder_path = "archive/test/"
class_names = [
    ['surprised', 'shocked', 'spooked'],
    ['angry'],
    ['fearful'],
    ['happy', 'smiling'],
    ['neutral', 'bored'],
    ['sad', 'depressed'],
    ['disgusted'],
]
directory_names = ['surprise', 'angry', 'fear', 'happy', 'neutral', 'sad', 'disgust']
templates = [
    'a photo of a {} looking face.',
    'a photo of a face showing the emotion: {}.',
    'a photo of a face looking {}.',
    'a face that looks {}.',
    'they look {}.',
    'look at how {} they are.',
]

flat_class_names = list(itertools.chain.from_iterable(class_names))



formatted_output = []
for emotion in flat_class_names:
  for template in templates:
    formatted_output.append(template.format(emotion))

all_templates = formatted_output


class_value = []
for i in range(len(class_names)):
  class_value.append([i] * len(class_names[i]) * len(templates))
class_value = list(itertools.chain.from_iterable(class_value))


def analyiseData2(n):
  confusion_matrix = np.zeros((len(class_names), len(class_names)))
  correct_answer = 0
  correct_classifications = 0
  incorrect_classifications = 0
  text = tokenizer(all_templates).to('cuda')
  for class_name in directory_names:
    m = 0
    arr = os.listdir(os.path.join(dataset_folder_path, class_name))
    for image_name in arr:
      m += 1
      values = np.array(analyise2(image_name, text, dataset_folder_path + class_name + "/"))
      # print(values)
      # print("Actual prediction: ", np.argmax(values))
      # print("Predicted: ", class_value[np.argmax(values)])
      # print("Correct: ", correct_answer)
      # time.sleep(2)
      #print(list(zip(class_names, values)))
      if class_value[np.argmax(values)] == correct_answer:
          correct_classifications += 1
      else:
          incorrect_classifications += 1
      print("Currently classifying " + image_name, "  Done: ", (correct_classifications + incorrect_classifications), "/", n * len(class_names), " Current accuracy: ", correct_classifications/(correct_classifications + incorrect_classifications))
      confusion_matrix[correct_answer][class_value[np.argmax(values)]] += 1
      clear_output(wait=True)
      if n == m:
        break
    #print(correct_answers)
    #print(incorrect_answers)
    correct_answer += 1
  print("Accuracy on ", n, " images from every class: ", correct_classifications/(correct_classifications + incorrect_classifications))
  print(confusion_matrix)

In [24]:
analyiseData2(100)

Accuracy on  100  images from every class:  0.38285714285714284
[[89.  1.  2.  6.  1.  1.  0.]
 [31. 23. 29.  2.  4. 11.  0.]
 [36.  6. 18.  2.  6. 32.  0.]
 [12.  0.  5. 75.  4.  4.  0.]
 [27.  2. 26.  5. 16. 24.  0.]
 [24.  7. 11.  3.  8. 47.  0.]
 [40. 28. 12.  6.  9.  5.  0.]]


In [ ]:
analyiseData2(-1)

Accuracy on  -1  images from every class:  0.4449847179772159
[[7.220e+02 9.000e+00 3.200e+01 6.600e+01 1.000e+01 1.200e+01 0.000e+00]
 [3.260e+02 2.440e+02 1.950e+02 4.900e+01 3.600e+01 1.070e+02 1.000e+00]
 [4.270e+02 5.200e+01 1.600e+02 5.100e+01 3.400e+01 3.000e+02 0.000e+00]
 [2.270e+02 2.000e+00 1.280e+02 1.292e+03 8.800e+01 3.600e+01 1.000e+00]
 [2.440e+02 4.200e+01 2.980e+02 6.800e+01 2.150e+02 3.650e+02 1.000e+00]
 [2.500e+02 8.600e+01 2.220e+02 4.700e+01 7.200e+01 5.700e+02 0.000e+00]
 [4.600e+01 3.300e+01 1.200e+01 6.000e+00 9.000e+00 5.000e+00 0.000e+00]]


Dobiven je ukupan accuracy od 44.5%, na službenoj stranici je dobiven accuracy od 47.3% pa vidimo da je ovo dosta dobro. 

In [17]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


classes = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck',
]
templates = [
    'a photo of a {}.',
    'a blurry photo of a {}.',
    'a black and white photo of a {}.',
    'a low contrast photo of a {}.',
    'a high contrast photo of a {}.',
    'a bad photo of a {}.',
    'a good photo of a {}.',
    'a photo of a small {}.',
    'a photo of a big {}.',
    'a photo of the {}.',
    'a blurry photo of the {}.',
    'a black and white photo of the {}.',
    'a low contrast photo of the {}.',
    'a high contrast photo of the {}.',
    'a bad photo of the {}.',
    'a good photo of the {}.',
    'a photo of the small {}.',
    'a photo of the big {}.',
]

formatted_output = []
for clas in classes:
  for template in templates:
    formatted_output.append(template.format(clas))

all_templates = formatted_output
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
batch_size = len(testset)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


to_pil = transforms.ToPILImage()

def analyiseData3(n, images, labels):
  classified = 0
  correct_classifications = 0
  incorrect_classifications = 0
  text = tokenizer(all_templates).to('cuda')
  for i in range(n):
    classified += 1
    img = images[i]/2 + 0.5
    img = to_pil(img)
    retarr = np.array(analyise3(img, text))
    if(labels[i] == np.argmax(retarr) // len(templates)):
      correct_classifications += 1
    else:
      incorrect_classifications += 1
      print("Currently classifying " + image_name, "  Done: ", (correct_classifications + incorrect_classifications), "/", n * len(class_names), " Current accuracy: ", correct_classifications/(correct_classifications + incorrect_classifications))
    clear_output(wait=True)
  print("Accuracy on batch size of ", n, " ", correct_classifications/(correct_classifications + incorrect_classifications))

Files already downloaded and verified


In [18]:
%pip install matplotlib
import matplotlib
import matplotlib.pyplot as plt
dataiter = iter(testloader)
images, labels = next(dataiter)

Note: you may need to restart the kernel to use updated packages.


In [19]:
analyiseData3(batch_size, images, labels)

Accuracy on batch size of  10000   0.889


Dobivamo accuracy od 88.9% dok je u paperu dobiveno 91.3% što je zadovoljavajuće 

In [20]:
%pip install pandas
%pip install openpyxl
import pandas as pd
df = pd.read_excel('stanford_cars_with_class_names.xlsx', sheet_name=1)
car_classes = [
    'AM General Hummer SUV 2000',
    'Acura RL Sedan 2012',
    'Acura TL Sedan 2012',
    'Acura TL Type-S 2008',
    'Acura TSX Sedan 2012',
    'Acura Integra Type R 2001',
    'Acura ZDX Hatchback 2012',
    'Aston Martin V8 Vantage Convertible 2012',
    'Aston Martin V8 Vantage Coupe 2012',
    'Aston Martin Virage Convertible 2012',
    'Aston Martin Virage Coupe 2012',
    'Audi RS 4 Convertible 2008',
    'Audi A5 Coupe 2012',
    'Audi TTS Coupe 2012',
    'Audi R8 Coupe 2012',
    'Audi V8 Sedan 1994',
    'Audi 100 Sedan 1994',
    'Audi 100 Wagon 1994',
    'Audi TT Hatchback 2011',
    'Audi S6 Sedan 2011',
    'Audi S5 Convertible 2012',
    'Audi S5 Coupe 2012',
    'Audi S4 Sedan 2012',
    'Audi S4 Sedan 2007',
    'Audi TT RS Coupe 2012',
    'BMW ActiveHybrid 5 Sedan 2012',
    'BMW 1 Series Convertible 2012',
    'BMW 1 Series Coupe 2012',
    'BMW 3 Series Sedan 2012',
    'BMW 3 Series Wagon 2012',
    'BMW 6 Series Convertible 2007',
    'BMW X5 SUV 2007',
    'BMW X6 SUV 2012',
    'BMW M3 Coupe 2012',
    'BMW M5 Sedan 2010',
    'BMW M6 Convertible 2010',
    'BMW X3 SUV 2012',
    'BMW Z4 Convertible 2012',
    'Bentley Continental Supersports Conv. Convertible 2012',
    'Bentley Arnage Sedan 2009',
    'Bentley Mulsanne Sedan 2011',
    'Bentley Continental GT Coupe 2012',
    'Bentley Continental GT Coupe 2007',
    'Bentley Continental Flying Spur Sedan 2007',
    'Bugatti Veyron 16.4 Convertible 2009',
    'Bugatti Veyron 16.4 Coupe 2009',
    'Buick Regal GS 2012',
    'Buick Rainier SUV 2007',
    'Buick Verano Sedan 2012',
    'Buick Enclave SUV 2012',
    'Cadillac CTS-V Sedan 2012',
    'Cadillac SRX SUV 2012',
    'Cadillac Escalade EXT Crew Cab 2007',
    'Chevrolet Silverado 1500 Hybrid Crew Cab 2012',
    'Chevrolet Corvette Convertible 2012',
    'Chevrolet Corvette ZR1 2012',
    'Chevrolet Corvette Ron Fellows Edition Z06 2007',
    'Chevrolet Traverse SUV 2012',
    'Chevrolet Camaro Convertible 2012',
    'Chevrolet HHR SS 2010',
    'Chevrolet Impala Sedan 2007',
    'Chevrolet Tahoe Hybrid SUV 2012',
    'Chevrolet Sonic Sedan 2012',
    'Chevrolet Express Cargo Van 2007',
    'Chevrolet Avalanche Crew Cab 2012',
    'Chevrolet Cobalt SS 2010',
    'Chevrolet Malibu Hybrid Sedan 2010',
    'Chevrolet TrailBlazer SS 2009',
    'Chevrolet Silverado 2500HD Regular Cab 2012',
    'Chevrolet Silverado 1500 Classic Extended Cab 2007',
    'Chevrolet Express Van 2007',
    'Chevrolet Monte Carlo Coupe 2007',
    'Chevrolet Malibu Sedan 2007',
    'Chevrolet Silverado 1500 Extended Cab 2012',
    'Chevrolet Silverado 1500 Regular Cab 2012',
    'Chrysler Aspen SUV 2009',
    'Chrysler Sebring Convertible 2010',
    'Chrysler Town and Country Minivan 2012',
    'Chrysler 300 SRT-8 2010',
    'Chrysler Crossfire Convertible 2008',
    'Chrysler PT Cruiser Convertible 2008',
    'Daewoo Nubira Wagon 2002',
    'Dodge Caliber Wagon 2012',
    'Dodge Caliber Wagon 2007',
    'Dodge Caravan Minivan 1997',
    'Dodge Ram Pickup 3500 Crew Cab 2010',
    'Dodge Ram Pickup 3500 Quad Cab 2009',
    'Dodge Sprinter Cargo Van 2009',
    'Dodge Journey SUV 2012',
    'Dodge Dakota Crew Cab 2010',
    'Dodge Dakota Club Cab 2007',
    'Dodge Magnum Wagon 2008',
    'Dodge Challenger SRT8 2011',
    'Dodge Durango SUV 2012',
    'Dodge Durango SUV 2007',
    'Dodge Charger Sedan 2012',
    'Dodge Charger SRT-8 2009',
    'Eagle Talon Hatchback 1998',
    'FIAT 500 Abarth 2012',
    'FIAT 500 Convertible 2012',
    'Ferrari FF Coupe 2012',
    'Ferrari California Convertible 2012',
    'Ferrari 458 Italia Convertible 2012',
    'Ferrari 458 Italia Coupe 2012',
    'Fisker Karma Sedan 2012',
    'Ford F-450 Super Duty Crew Cab 2012',
    'Ford Mustang Convertible 2007',
    'Ford Freestar Minivan 2007',
    'Ford Expedition EL SUV 2009',
    'Ford Edge SUV 2012',
    'Ford Ranger SuperCab 2011',
    'Ford GT Coupe 2006',
    'Ford F-150 Regular Cab 2012',
    'Ford F-150 Regular Cab 2007',
    'Ford Focus Sedan 2007',
    'Ford E-Series Wagon Van 2012',
    'Ford Fiesta Sedan 2012',
    'GMC Terrain SUV 2012',
    'GMC Savana Van 2012',
    'GMC Yukon Hybrid SUV 2012',
    'GMC Acadia SUV 2012',
    'GMC Canyon Extended Cab 2012',
    'Geo Metro Convertible 1993',
    'HUMMER H3T Crew Cab 2010',
    'HUMMER H2 SUT Crew Cab 2009',
    'Honda Odyssey Minivan 2012',
    'Honda Odyssey Minivan 2007',
    'Honda Accord Coupe 2012',
    'Honda Accord Sedan 2012',
    'Hyundai Veloster Hatchback 2012',
    'Hyundai Santa Fe SUV 2012',
    'Hyundai Tucson SUV 2012',
    'Hyundai Veracruz SUV 2012',
    'Hyundai Sonata Hybrid Sedan 2012',
    'Hyundai Elantra Sedan 2007',
    'Hyundai Accent Sedan 2012',
    'Hyundai Genesis Sedan 2012',
    'Hyundai Sonata Sedan 2012',
    'Hyundai Elantra Touring Hatchback 2012',
    'Hyundai Azera Sedan 2012',
    'Infiniti G Coupe IPL 2012',
    'Infiniti QX56 SUV 2011',
    'Isuzu Ascender SUV 2008',
    'Jaguar XK XKR 2012',
    'Jeep Patriot SUV 2012',
    'Jeep Wrangler SUV 2012',
    'Jeep Liberty SUV 2012',
    'Jeep Grand Cherokee SUV 2012',
    'Jeep Compass SUV 2012',
    'Lamborghini Reventon Coupe 2008',
    'Lamborghini Aventador Coupe 2012',
    'Lamborghini Gallardo LP 570-4 Superleggera 2012',
    'Lamborghini Diablo Coupe 2001',
    'Land Rover Range Rover SUV 2012',
    'Land Rover LR2 SUV 2012',
    'Lincoln Town Car Sedan 2011',
    'MINI Cooper Roadster Convertible 2012',
    'Maybach Landaulet Convertible 2012',
    'Mazda Tribute SUV 2011',
    'McLaren MP4-12C Coupe 2012',
    'Mercedes-Benz 300-Class Convertible 1993',
    'Mercedes-Benz C-Class Sedan 2012',
    'Mercedes-Benz SL-Class Coupe 2009',
    'Mercedes-Benz E-Class Sedan 2012',
    'Mercedes-Benz S-Class Sedan 2012',
    'Mercedes-Benz Sprinter Van 2012',
    'Mitsubishi Lancer Sedan 2012',
    'Nissan Leaf Hatchback 2012',
    'Nissan NV Passenger Van 2012',
    'Nissan Juke Hatchback 2012',
    'Nissan 240SX Coupe 1998',
    'Plymouth Neon Coupe 1999',
    'Porsche Panamera Sedan 2012',
    'Ram C/V Cargo Van Minivan 2012',
    'Rolls-Royce Phantom Drophead Coupe Convertible 2012',
    'Rolls-Royce Ghost Sedan 2012',
    'Rolls-Royce Phantom Sedan 2012',
    'Scion xD Hatchback 2012',
    'Spyker C8 Convertible 2009',
    'Spyker C8 Coupe 2009',
    'Suzuki Aerio Sedan 2007',
    'Suzuki Kizashi Sedan 2012',
    'Suzuki SX4 Hatchback 2012',
    'Suzuki SX4 Sedan 2012',
    'Tesla Model S Sedan 2012',
    'Toyota Sequoia SUV 2012',
    'Toyota Camry Sedan 2012',
    'Toyota Corolla Sedan 2012',
    'Toyota 4Runner SUV 2012',
    'Volkswagen Golf Hatchback 2012',
    'Volkswagen Golf Hatchback 1991',
    'Volkswagen Beetle Hatchback 2012',
    'Volvo C30 Hatchback 2012',
    'Volvo 240 Sedan 1993',
    'Volvo XC90 SUV 2007',
    'smart fortwo Convertible 2012',
]
car_annos = np.array(df["class"])
car_templates = [
    'a photo of a {}.',
    'a photo of the {}.',
    'a photo of my {}.',
    'i love my {}!',
    'a photo of my dirty {}.',
    'a photo of my clean {}.',
    'a photo of my new {}.',
    'a photo of my old {}.',
]
car_formatted_output = []
for clas in car_classes:
  for template in car_templates:
    car_formatted_output.append(template.format(clas))

car_values = []
for i in range(len(car_classes)):
  for _ in range(len(car_templates)):
    car_values.append(i)
car_template_class = list(zip(car_formatted_output, car_values))


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [21]:
import torch
import os
import numpy as np
from IPython.display import clear_output


model, preprocess = clip.load("ViT-B/32")
model.to('cuda')
model.eval()
tokenizer = open_clip.get_tokenizer('ViT-B-32')
dataset_folder_path = "cars_test/cars_test/"

def analyiseData3(n):
  m = 0
  correct_answer = 0
  correct_classifications = 0
  incorrect_classifications = 0
  text = tokenizer(car_formatted_output).to('cuda')
  arr = os.listdir(os.path.join(dataset_folder_path))
  np.sort(arr)
  print(len(arr))
  for _ in arr:
    print(str(m + 1).zfill(5) + ".jpg")
    image_name = str(m + 1).zfill(5) + ".jpg"
    correct_class = car_annos[m] - 1
    print(image_name)
    print("Correct class: ", correct_class, car_classes[correct_class])
    values = np.array(analyise2_no_resize(image_name, text, dataset_folder_path))
    #print(list(zip(class_names, values)))
    print("Predicted class: ", car_values[np.argmax(values)], car_classes[car_values[np.argmax(values)]])
    if car_values[np.argmax(values)] == correct_class:
        correct_classifications += 1
    else:
        incorrect_classifications += 1
    print(values[correct_class])
    clear_output(wait=True)
    print("Currently classifying " + image_name, "  Done: ", (correct_classifications + incorrect_classifications), "/", len(arr), " Current accuracy: ", correct_classifications/(correct_classifications + incorrect_classifications))
    m += 1
    if n == m:
      break
  #print(correct_answers)
  #print(incorrect_answers)
  correct_answer += 1
  print("Accuracy on ", n, " images from every class: ", correct_classifications/(correct_classifications + incorrect_classifications))

In [15]:
analyiseData3(8041)

Currently classifying 08041.jpg   Done:  8041 / 8041  Current accuracy:  0.5924636239273722
Accuracy on  8041  images from every class:  0.5924636239273722


Točnost koju sam dobio je 59.24% što se podudara s benchmark točnosti od 59.4%